### Tên : Huỳnh Ngọc Công Danh
### MSSV : 19521322

In [1]:
from keras.layers import Conv2D,Input,Dense,MaxPool2D,Flatten,AveragePooling2D,Add
from keras.models import load_model,Model
class MyCNN:
  def __init__(self):
    self.model = None
  def build(self,input_dim):
    input = Input(shape=input_dim)
    c1 = Conv2D(8,(3,3),padding='same',activation='relu')(input)
    s2 = MaxPool2D(strides=(2,2),padding='same',pool_size=(2,2))(c1)

    c3 = Conv2D(8,(3,3),padding='same',activation='relu')(s2)
    c4 = Conv2D(8,(3,3),padding='same',activation='relu')(c3)
    C_34 = Add()([c3, c4])
    c5 = Conv2D(16,(3,3),padding='same',activation='relu',strides=(2,2))(C_34)
    c6 = Conv2D(16,(3,3),padding='same',activation='relu')(c5)
    c7 = Conv2D(16,(3,3),padding='same',activation='relu')(c6)
    C_67 = Add()([c6,c7])
    A8 = AveragePooling2D()(C_67)
    flat_a8 = Flatten()(A8)
    F9 = Dense(400,activation='sigmoid',use_bias=False)(flat_a8)
    output = Dense(10,activation='softmax',use_bias=False)(F9)
    
    self.model = Model(input,output)
    self.model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  def train(self, x_train, y_train):
    self.model.fit(x_train, y_train, epochs = 50, batch_size = 128)
  def save(self,model_file):
    self.model.save(model_file)
  def summary(self):
    self.model.summary()
  def load(self,model_file):
    self.model = load_model(model_file)
  def predict(self,x_test):
    return self.model.predict(x_test)
  def evaluate(self,x_test,y_test):
    print("Evaluate on test data")
    _, acc = self.model.evaluate(x_test, y_test, batch_size=32)
    print("accurac_score :", acc)


In [2]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [4]:
import numpy as np

def onehot(y):
  oh = np.zeros((y.shape[0], 10))
  for i in range(y.shape[0]):
    oh[i, int(y[i])]=1
  return oh
from sklearn import preprocessing
import numpy as np
y_train_onehot = onehot(y_train)
y_train_onehot.shape

y_test_onehot = onehot(y_test)
y_test_onehot.shape

(10000, 10)

In [5]:
cnn = MyCNN()
cnn.build((28, 28, 1))
cnn.summary()
# cnn.train(x_train, y_train_onehot)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 8)    80          input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 14, 14, 8)    0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 14, 14, 8)    584         max_pooling2d[0][0]              
______________________________________________________________________________________________

In [7]:
cnn.train(x_train, y_train_onehot)


Epoch 1/50
170/469 [=========>....................] - ETA: 26s - loss: 0.0025 - accuracy: 0.9967

KeyboardInterrupt: ignored

In [10]:
cnn.save("mycnn.h5")

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [12]:
new_cnn = MyCNN()
new_cnn.load("mycnn.h5")
new_cnn.evaluate(x_test,y_test_onehot)

Evaluate on test data
313/313 [==============================] - 5s 15ms/step - loss: 0.0064 - accuracy: 0.9907
accurac_score : 0.9907000064849854
